In [5]:
from DataParser import DataParser
import numpy as np
import scipy.sparse as sps
from sklearn.model_selection import KFold

from Data_manager.split_functions.split_train_validation_random_holdout import \
split_train_in_two_percentage_global_sample

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from SimpleEvaluator import evaluator
from Base.Evaluation.Evaluator import EvaluatorHoldout

In [6]:
from Base.NonPersonalizedRecommender import TopPop, Random
from Hybrid.HybridCombinationSearchCV import HybridCombinationSearchCV
from KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
from KNN.UserKNNCFRecommender import UserKNNCFRecommender
from KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from KNN.SpecialItemKNNCBFRecommender import SpecialItemKNNCBFRecommender
from MatrixFactorization.IALSRecommender import IALSRecommender
from SLIM_BPR.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from SLIM_ElasticNet.SLIMElasticNetRecommender import SLIMElasticNetRecommender
from SLIM_ElasticNet.SSLIM_ElasticNet import SSLIMElasticNet
from GraphBased.P3alphaRecommender import P3alphaRecommender
from GraphBased.RP3betaRecommender import RP3betaRecommender
from MatrixFactorization.Cython.MatrixFactorization_Cython import MatrixFactorization_BPR_Cython, MatrixFactorization_FunkSVD_Cython
from MatrixFactorization.PureSVDRecommender import PureSVDRecommender, PureSVDItemRecommender
from MatrixFactorization.NMFRecommender import NMFRecommender
from KNN.ItemKNN_CBF_CF import ItemKNN_CBF_CF

In [7]:
seed = 1666
parser = DataParser()

URM_all = parser.get_URM_all()
ICM_all = parser.get_ICM_all()

#URM_train, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage=0.85, seed=seed)

In [8]:
rp3b = (RP3betaRecommender, {'topK': 926, 'alpha': 0.4300109351916609, 'beta': 0.01807360750913967, 'normalize_similarity': False})
p3a = (P3alphaRecommender, {'topK': 575, 'alpha': 0.48009885897470206, 'normalize_similarity': False})
icf = (ItemKNNCFRecommender, {'topK': 1000, 'shrink': 1000, 'similarity': 'cosine', 'normalize': True, 'feature_weighting': 'TF-IDF'})
ucf = (UserKNNCFRecommender, {'topK': 163, 'shrink': 846, 'similarity': 'cosine', 'normalize': True, 'feature_weighting': 'TF-IDF'})
icb = (ItemKNNCBFRecommender, {'topK': 164, 'shrink': 8, 'similarity': 'jaccard', 'normalize': True})
sbpr = (SLIM_BPR_Cython, {'topK': 1000, 'epochs': 130, 'symmetric': False, 'sgd_mode': 'adam', 'lambda_i': 1e-05, 'lambda_j': 1e-05, 'learning_rate': 0.0001})
icfcb = (ItemKNN_CBF_CF, {'topK': 1000, 'shrink': 1000, 'similarity': 'asymmetric', 'normalize': True, 'asymmetric_alpha': 0.241892724784089, 'feature_weighting': 'TF-IDF', 'icm_weight': 1.0})
sslim = (SSLIMElasticNet, {'beta': 0.567288665094892, 'topK': 1000, 'l1_ratio': 1e-05, 'alpha': 0.001})
psvd = (PureSVDRecommender, {'num_factors': 350})
icbsup = (SpecialItemKNNCBFRecommender, {'topK': 1000, 'shrink': 1000, 'similarity': 'cosine', 'normalize': True, 'feature_weighting': 'BM25'})

In [9]:
combo_labels ={
    'combo1': 'icb, icf, p3a',
    'combo2': 'cb, icf, rp3b',
    'combo3': 'cb, rp3b, p3a',
    'combo4': 'icb, ucf, p3a', 
    'combo5': 'icb, ucf, rp3b',
    'combo6': 'icb, icfcb, icf',
    'combo7': 'icb, icfcb, p3a',
    'combo8': 'icb, icfcb, rp3b',
    'combo9': 'icb, icfcb, ucf',
    'combo10': 'icb, icfcb, sbpr',
    'combo11': 'icbsup, icfcb, icf',
    'combo12': 'icbsup, icfcb, p3a',
    'combo13': 'icbsup, icfcb, rp3b',
    'combo14': 'icbsup, icfcb, ucf',
    'combo15': 'icbsup, icfcb, sbpr',
    'combo16': 'icbsup, icf, p3a',
    'combo17': 'icbsup, icfcb, rp3b',
    'combo18': 'icbsup, icf, ucf',
    'combo19': 'icbsup, icf, sbpr',
    'combo20': 'icbsup, p3a, rp3b',
    'combo21': 'icbsuo, ucf, p3a',
    'combo22': 'icbsup, ucf, rp3b',
    'combo23': 'icbsup, ucf, sbpr',
    'combo24': 'icbsup, sbpr, sslim',
    'combo25': 'icbsup, p3a, sslim',
    'combo26': 'icbsup, ucf, sslim',
    'combo27': 'icbsup, icf, sslim',
    'combo28': 'icbsup, icfcb, sslim',
    'combo29': 'icbsup, rp3b, sslim',
    'combo30': 'icb, sbpr, sslim',
    'combo31': 'icb, rp3b, sslim',
    'combo32': 'icb, p3a, sslim',
    'combo33': 'rp3b',
    'combo34': 'p3a',
    'combo35': 'icf',
    'combo36': 'ucf',
    'combo37': 'icb',
    'combo38': 'sbpr',
    'combo39': 'icfcb',
    'combo40': 'sslim',
    'combo41': 'psvd',
    'combo42': 'icbsup',
}

In [10]:
k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=seed)

URM_list = []
URM_test_list = []
evaluator_list = []

shape = URM_all.shape
indptr = URM_all.indptr
indices = URM_all.indices
data = URM_all.data

for train_index, test_index in kf.split(data):
    data_train = np.ones(data.shape)
    data_test = np.ones(data.shape)
    data_train[test_index] = 0
    data_test[train_index] = 0
    kf_train = sps.csr_matrix((data_train, indices, indptr), shape=shape).copy()
    kf_test = sps.csr_matrix((data_test, indices, indptr), shape=shape).copy()
    kf_train.eliminate_zeros()
    kf_test.eliminate_zeros()
    URM_list.append(kf_train)
    URM_test_list.append(kf_test)
    evaluator_list.append(EvaluatorHoldout(kf_test, cutoff_list=[10]))

In [12]:
list_rec = [([icb, icf, p3a], {'alpha': 0.44569420382449637, 'l1_ratio': 0.6168742754254437}),
            ([icb, icf, rp3b], {'alpha': 0.3033751201293658, 'l1_ratio': 0.7822655054333012}),
            ([icb, icf, ucf], {'alpha': 0.48973465405541533, 'l1_ratio': 0.3567553681370489}),
            ([icb, rp3b, p3a], {'alpha': 0.6171935935379741, 'l1_ratio': 0.4090970817953153}),
            ([icb, ucf, p3a], {'alpha': 0.6310254748569911, 'l1_ratio': 0.42318219644561506}),
            ([icb, ucf, rp3b], {'alpha': 0.5388201552532741, 'l1_ratio': 0.35597718538152645}),
            ([icb, icfcb, icf], {'alpha': 0.9457062373092435, 'l1_ratio': 0.08675668683308081}),
            ([icb, icfcb, p3a], {'alpha': 0.6904008032335807, 'l1_ratio': 0.23371348584724985}),
            ([icb, icfcb, rp3b], {'alpha': 0.8129428861277737, 'l1_ratio': 0.14069122456123387}),
            ([icb, icfcb, ucf], {'alpha': 0.8129428861277737, 'l1_ratio': 0.14069122456123387}),
            ([icb, icfcb, sbpr], {'alpha': 0.8991751672246813, 'l1_ratio': 0.11874637825106651}),
            ([icbsup, icfcb, icf], {'alpha': 0.8122492419947548, 'l1_ratio': 0.1448868067763832}),
            ([icbsup, icfcb, p3a], {'alpha': 0.6094679138906033, 'l1_ratio': 0.41715098578193194}),
            ([icbsup, icfcb, rp3b], {'alpha': 0.5369911284211629, 'l1_ratio': 0.45078644454282}),
            ([icbsup, icfcb, ucf], {'alpha': 0.7020854096807992, 'l1_ratio': 0.20749665943340362}),
            ([icbsup, icfcb, sbpr], {'alpha': 0.7949204415356104, 'l1_ratio': 0.2705694962114903}),
            ([icbsup, icf, p3a], {'alpha': 0.4973334775913678, 'l1_ratio': 0.6790274919219236}),
            ([icbsup, icfcb, rp3b], {'alpha': 0.45931982254221104, 'l1_ratio': 0.552400663560699}),
            ([icbsup, icf, ucf], {'alpha': 0.5934342246244017, 'l1_ratio': 0.38793882442583466}),
            ([icbsup, icf, sbpr], {'alpha': 0.8170119826252383, 'l1_ratio': 0.33586027250410305}),
            ([icbsup, p3a, rp3b], {'alpha': 0.4001101186128191, 'l1_ratio': 0.9989082076640393}),
            ([icbsup, ucf, p3a], {'alpha': 0.627349023025726, 'l1_ratio': 0.5343474311262314}),
            ([icbsup, ucf, rp3b], {'alpha': 0.5574853559798421, 'l1_ratio': 0.5221328226891215}),
            ([icbsup, ucf, sbpr], {'alpha': 0.9033873534648698, 'l1_ratio': 0.30678626017844307}),
            ([icbsup, sbpr, sslim], {'alpha': 0.6194113348305054, 'l1_ratio': 0.6156379091138154}),
            ([icbsup, p3a, sslim], {'alpha': 0.7989350513339316, 'l1_ratio': 0.4461703935735907}),
            ([icbsup, ucf, sslim], {'alpha': 0.7362031685050584, 'l1_ratio': 0.3394266517525301}),
            ([icbsup, icf, sslim], {'alpha': 0.6383024036201149, 'l1_ratio': 0.5132363657816151}),
            ([icbsup, icfcb, sslim], {'alpha': 0.8162670192627814, 'l1_ratio': 0.26730373918567046}),
            ([icbsup, rp3b, sslim], {'alpha': 0.7881737847053779, 'l1_ratio': 0.400772981812612}),
            ([icb, sbpr, sslim], {'alpha': 0.5238332406148302, 'l1_ratio': 0.5907932058981844}),
            ([icb, rp3b, sslim], {'alpha': 0.7861575718386372, 'l1_ratio': 0.34264216397962627}),
            ([icb, p3a, sslim], {'alpha': 0.6325200648079097, 'l1_ratio': 0.4685932097360991}),
            rp3b,
            p3a,
            icf,
            ucf,
            icb,
            sbpr,
            icfcb,
            sslim,
            psvd,
            icbsup,
]

In [13]:
num_combo = len(list_rec)

In [16]:
f_ranges = [
    (0,3),
    (3,5),
    (5,10),
    (10,17),
    (17,30),
    (30,100),
    (100,200),
    (200,-1)
]

MAP_combo_per_group = []
cutoff = 10

for f_range in f_ranges:
    
    
    combo_dict={}
    for k in range (num_combo):
        r = []
        for i in range (len(URM_list)):
            URM_test_group = parser.filter_URM_test_by_range(URM_list[i], URM_test_list[i], f_range)
            evaluator_test = EvaluatorHoldout(URM_test_group, cutoff_list=[cutoff])
            if not isinstance(list_rec[k][0], list):
                rec = list_rec[k][0]
                try:
                    rec = rec(URM_all, ICM_all, verbose = False)
                except:
                    rec = rec(URM_all, verbose=False)
                try:
                    rec.load_model(f'stored_recommenders/seed_{str(seed)}_hybrid_search/{rec.RECOMMENDER_NAME}/', f'{str(seed)}_fold-{str(i)}')
                    print(f"{rec.RECOMMENDER_NAME} loaded. [seed={seed}, fold={i}]")
                except:
                    print(f"Fitting {rec.RECOMMENDER_NAME} ... [seed={seed}, fold={i}]")
                    rec.fit(**list_rec[k][1])
                    print(f"done.")
                    rec.save_model(f'stored_recommenders/seed_{str(seed)}_hybrid_search/{rec.RECOMMENDER_NAME}/', f'{str(seed)}_fold-{str(i)}')
            else:
                rec = HybridCombinationSearchCV(URM_list[i], ICM_all, list_rec[k][0], seed = seed, fold = i)
                rec.fit(**list_rec[k][1])

            results, _ = evaluator_test.evaluateRecommender(rec)
            r.append(results[cutoff]["MAP"])
        combo_dict[k]=np.average(r)

    MAP_combo_per_group.append(combo_dict)

HybridCombinationSearchCV: URM Detected 48 (0.60 %) cold users.
HybridCombinationSearchCV: URM Detected 2381 (9.17 %) cold items.
ItemKNNCBFRecommender loaded. [seed=1666, fold=0]
ItemKNNCFRecommender loaded. [seed=1666, fold=0]
P3alphaRecommender loaded. [seed=1666, fold=0]
EvaluatorHoldout: Processed 1144 ( 100.00% ) in 1.04 sec. Users per second: 1097
HybridCombinationSearchCV: URM Detected 68 (0.86 %) cold users.
HybridCombinationSearchCV: URM Detected 2439 (9.39 %) cold items.
ItemKNNCBFRecommender loaded. [seed=1666, fold=1]
ItemKNNCFRecommender loaded. [seed=1666, fold=1]
P3alphaRecommender loaded. [seed=1666, fold=1]
EvaluatorHoldout: Processed 1069 ( 100.00% ) in 0.97 sec. Users per second: 1101
HybridCombinationSearchCV: URM Detected 56 (0.70 %) cold users.
HybridCombinationSearchCV: URM Detected 2402 (9.25 %) cold items.
ItemKNNCBFRecommender loaded. [seed=1666, fold=2]
ItemKNNCFRecommender loaded. [seed=1666, fold=2]
P3alphaRecommender loaded. [seed=1666, fold=2]
EvaluatorH

EvaluatorHoldout: Processed 1132 ( 100.00% ) in 1.02 sec. Users per second: 1112
HybridCombinationSearchCV: URM Detected 65 (0.82 %) cold users.
HybridCombinationSearchCV: URM Detected 2338 (9.00 %) cold items.
ItemKNNCBFRecommender loaded. [seed=1666, fold=3]
RP3betaRecommender loaded. [seed=1666, fold=3]
P3alphaRecommender loaded. [seed=1666, fold=3]
EvaluatorHoldout: Processed 1091 ( 100.00% ) in 0.99 sec. Users per second: 1106
HybridCombinationSearchCV: URM Detected 77 (0.97 %) cold users.
HybridCombinationSearchCV: URM Detected 2398 (9.23 %) cold items.
ItemKNNCBFRecommender loaded. [seed=1666, fold=4]
RP3betaRecommender loaded. [seed=1666, fold=4]
P3alphaRecommender loaded. [seed=1666, fold=4]
EvaluatorHoldout: Processed 1059 ( 100.00% ) in 0.96 sec. Users per second: 1105
HybridCombinationSearchCV: URM Detected 48 (0.60 %) cold users.
HybridCombinationSearchCV: URM Detected 2381 (9.17 %) cold items.
ItemKNNCBFRecommender loaded. [seed=1666, fold=0]
UserKNNCFRecommender loaded. 

ItemKNN_CBF_CF_Recommender loaded. [seed=1666, fold=0]
P3alphaRecommender loaded. [seed=1666, fold=0]
EvaluatorHoldout: Processed 1144 ( 100.00% ) in 1.16 sec. Users per second: 989
HybridCombinationSearchCV: URM Detected 68 (0.86 %) cold users.
HybridCombinationSearchCV: URM Detected 2439 (9.39 %) cold items.
ItemKNNCBFRecommender loaded. [seed=1666, fold=1]
ItemKNN_CBF_CF_Recommender loaded. [seed=1666, fold=1]
P3alphaRecommender loaded. [seed=1666, fold=1]
EvaluatorHoldout: Processed 1069 ( 100.00% ) in 1.03 sec. Users per second: 1035
HybridCombinationSearchCV: URM Detected 56 (0.70 %) cold users.
HybridCombinationSearchCV: URM Detected 2402 (9.25 %) cold items.
ItemKNNCBFRecommender loaded. [seed=1666, fold=2]
ItemKNN_CBF_CF_Recommender loaded. [seed=1666, fold=2]
P3alphaRecommender loaded. [seed=1666, fold=2]
EvaluatorHoldout: Processed 1132 ( 100.00% ) in 1.11 sec. Users per second: 1016
HybridCombinationSearchCV: URM Detected 65 (0.82 %) cold users.
HybridCombinationSearchCV: U

ItemKNNCBFRecommender loaded. [seed=1666, fold=3]
ItemKNN_CBF_CF_Recommender loaded. [seed=1666, fold=3]
SLIM_BPR_Recommender loaded. [seed=1666, fold=3]
EvaluatorHoldout: Processed 1091 ( 100.00% ) in 1.04 sec. Users per second: 1053
HybridCombinationSearchCV: URM Detected 77 (0.97 %) cold users.
HybridCombinationSearchCV: URM Detected 2398 (9.23 %) cold items.
ItemKNNCBFRecommender loaded. [seed=1666, fold=4]
ItemKNN_CBF_CF_Recommender loaded. [seed=1666, fold=4]
SLIM_BPR_Recommender loaded. [seed=1666, fold=4]
EvaluatorHoldout: Processed 1059 ( 100.00% ) in 0.99 sec. Users per second: 1065
HybridCombinationSearchCV: URM Detected 48 (0.60 %) cold users.
HybridCombinationSearchCV: URM Detected 2381 (9.17 %) cold items.
Special-ItemKNNCBFRec loaded. [seed=1666, fold=0]
ItemKNN_CBF_CF_Recommender loaded. [seed=1666, fold=0]
ItemKNNCFRecommender loaded. [seed=1666, fold=0]
EvaluatorHoldout: Processed 1144 ( 100.00% ) in 1.11 sec. Users per second: 1031
HybridCombinationSearchCV: URM Dete

Special-ItemKNNCBFRec loaded. [seed=1666, fold=1]
ItemKNN_CBF_CF_Recommender loaded. [seed=1666, fold=1]
UserKNNCFRecommender loaded. [seed=1666, fold=1]
EvaluatorHoldout: Processed 1069 ( 100.00% ) in 1.04 sec. Users per second: 1033
HybridCombinationSearchCV: URM Detected 56 (0.70 %) cold users.
HybridCombinationSearchCV: URM Detected 2402 (9.25 %) cold items.
Special-ItemKNNCBFRec loaded. [seed=1666, fold=2]
ItemKNN_CBF_CF_Recommender loaded. [seed=1666, fold=2]
UserKNNCFRecommender loaded. [seed=1666, fold=2]
EvaluatorHoldout: Processed 1132 ( 100.00% ) in 1.12 sec. Users per second: 1014
HybridCombinationSearchCV: URM Detected 65 (0.82 %) cold users.
HybridCombinationSearchCV: URM Detected 2338 (9.00 %) cold items.
Special-ItemKNNCBFRec loaded. [seed=1666, fold=3]
ItemKNN_CBF_CF_Recommender loaded. [seed=1666, fold=3]
UserKNNCFRecommender loaded. [seed=1666, fold=3]
EvaluatorHoldout: Processed 1091 ( 100.00% ) in 1.06 sec. Users per second: 1031
HybridCombinationSearchCV: URM Dete

...

P3alphaRecommender loaded. [seed=1666, fold=3]
RP3betaRecommender loaded. [seed=1666, fold=3]
EvaluatorHoldout: Processed 27 ( 100.00% ) in 0.12 sec. Users per second: 232
HybridCombinationSearchCV: URM Detected 77 (0.97 %) cold users.
HybridCombinationSearchCV: URM Detected 2398 (9.23 %) cold items.
Special-ItemKNNCBFRec loaded. [seed=1666, fold=4]
P3alphaRecommender loaded. [seed=1666, fold=4]
RP3betaRecommender loaded. [seed=1666, fold=4]
EvaluatorHoldout: Processed 29 ( 100.00% ) in 0.13 sec. Users per second: 232
HybridCombinationSearchCV: URM Detected 48 (0.60 %) cold users.
HybridCombinationSearchCV: URM Detected 2381 (9.17 %) cold items.
Special-ItemKNNCBFRec loaded. [seed=1666, fold=0]
UserKNNCFRecommender loaded. [seed=1666, fold=0]
P3alphaRecommender loaded. [seed=1666, fold=0]
EvaluatorHoldout: Processed 28 ( 100.00% ) in 0.10 sec. Users per second: 269
HybridCombinationSearchCV: URM Detected 68 (0.86 %) cold users.
HybridCombinationSearchCV: URM Detected 2439 (9.39 %) cold

Special-ItemKNNCBFRec loaded. [seed=1666, fold=1]
SLIM_BPR_Recommender loaded. [seed=1666, fold=1]
S-SLIMElasticNet loaded. [seed=1666, fold=1]
EvaluatorHoldout: Processed 28 ( 100.00% ) in 0.26 sec. Users per second: 106
HybridCombinationSearchCV: URM Detected 56 (0.70 %) cold users.
HybridCombinationSearchCV: URM Detected 2402 (9.25 %) cold items.
Special-ItemKNNCBFRec loaded. [seed=1666, fold=2]
SLIM_BPR_Recommender loaded. [seed=1666, fold=2]
S-SLIMElasticNet loaded. [seed=1666, fold=2]
EvaluatorHoldout: Processed 28 ( 100.00% ) in 0.26 sec. Users per second: 108
HybridCombinationSearchCV: URM Detected 65 (0.82 %) cold users.
HybridCombinationSearchCV: URM Detected 2338 (9.00 %) cold items.
Special-ItemKNNCBFRec loaded. [seed=1666, fold=3]
SLIM_BPR_Recommender loaded. [seed=1666, fold=3]
S-SLIMElasticNet loaded. [seed=1666, fold=3]
EvaluatorHoldout: Processed 27 ( 100.00% ) in 0.26 sec. Users per second: 105
HybridCombinationSearchCV: URM Detected 77 (0.97 %) cold users.
HybridComb

Special-ItemKNNCBFRec loaded. [seed=1666, fold=4]
ItemKNNCFRecommender loaded. [seed=1666, fold=4]
S-SLIMElasticNet loaded. [seed=1666, fold=4]
EvaluatorHoldout: Processed 29 ( 100.00% ) in 0.27 sec. Users per second: 107
HybridCombinationSearchCV: URM Detected 48 (0.60 %) cold users.
HybridCombinationSearchCV: URM Detected 2381 (9.17 %) cold items.
Special-ItemKNNCBFRec loaded. [seed=1666, fold=0]
ItemKNN_CBF_CF_Recommender loaded. [seed=1666, fold=0]
S-SLIMElasticNet loaded. [seed=1666, fold=0]
EvaluatorHoldout: Processed 28 ( 100.00% ) in 0.28 sec. Users per second: 99
HybridCombinationSearchCV: URM Detected 68 (0.86 %) cold users.
HybridCombinationSearchCV: URM Detected 2439 (9.39 %) cold items.
Special-ItemKNNCBFRec loaded. [seed=1666, fold=1]
ItemKNN_CBF_CF_Recommender loaded. [seed=1666, fold=1]
S-SLIMElasticNet loaded. [seed=1666, fold=1]
EvaluatorHoldout: Processed 28 ( 100.00% ) in 0.29 sec. Users per second: 97
HybridCombinationSearchCV: URM Detected 56 (0.70 %) cold users.


ItemKNNCBFRecommender loaded. [seed=1666, fold=2]
RP3betaRecommender loaded. [seed=1666, fold=2]
S-SLIMElasticNet loaded. [seed=1666, fold=2]
EvaluatorHoldout: Processed 28 ( 100.00% ) in 0.24 sec. Users per second: 115
HybridCombinationSearchCV: URM Detected 65 (0.82 %) cold users.
HybridCombinationSearchCV: URM Detected 2338 (9.00 %) cold items.
ItemKNNCBFRecommender loaded. [seed=1666, fold=3]
RP3betaRecommender loaded. [seed=1666, fold=3]
S-SLIMElasticNet loaded. [seed=1666, fold=3]
EvaluatorHoldout: Processed 27 ( 100.00% ) in 0.24 sec. Users per second: 112
HybridCombinationSearchCV: URM Detected 77 (0.97 %) cold users.
HybridCombinationSearchCV: URM Detected 2398 (9.23 %) cold items.
ItemKNNCBFRecommender loaded. [seed=1666, fold=4]
RP3betaRecommender loaded. [seed=1666, fold=4]
S-SLIMElasticNet loaded. [seed=1666, fold=4]
EvaluatorHoldout: Processed 29 ( 100.00% ) in 0.24 sec. Users per second: 120
HybridCombinationSearchCV: URM Detected 48 (0.60 %) cold users.
HybridCombinatio

ItemKNNCBFRecommender loaded. [seed=1666, fold=1]
EvaluatorHoldout: Processed 28 ( 100.00% ) in 0.05 sec. Users per second: 597
ItemKNNCBFRecommender loaded. [seed=1666, fold=2]
EvaluatorHoldout: Processed 28 ( 100.00% ) in 0.05 sec. Users per second: 597
ItemKNNCBFRecommender loaded. [seed=1666, fold=3]
EvaluatorHoldout: Processed 27 ( 100.00% ) in 0.03 sec. Users per second: 864
ItemKNNCBFRecommender loaded. [seed=1666, fold=4]
EvaluatorHoldout: Processed 29 ( 100.00% ) in 0.05 sec. Users per second: 619
SLIM_BPR_Recommender loaded. [seed=1666, fold=0]
EvaluatorHoldout: Processed 28 ( 100.00% ) in 0.06 sec. Users per second: 448
SLIM_BPR_Recommender loaded. [seed=1666, fold=1]
EvaluatorHoldout: Processed 28 ( 100.00% ) in 0.06 sec. Users per second: 448
SLIM_BPR_Recommender loaded. [seed=1666, fold=2]
EvaluatorHoldout: Processed 28 ( 100.00% ) in 0.05 sec. Users per second: 597
SLIM_BPR_Recommender loaded. [seed=1666, fold=3]
EvaluatorHoldout: Processed 27 ( 100.00% ) in 0.06 sec. Us

In [17]:
def check_if_present(key_list, el):
    for key_val, combo_val in key_list:
        if key_val == el[0] and combo_val == el[1]:
            return True
    return False

In [19]:
to_be_plotted = []
for i in range(0, len (f_ranges)):
    print(f"RANGE {f_ranges[i]}")
    maxp = 0
    old_maxp = 0
    difference = 0
    old_name = ''
    max_name=''
    max_isCombo=None
    old_isCombo=None
    max_key=None
    old_key=None
    
    combo_dict = MAP_combo_per_group[i]
    for key, value in combo_dict.items():
        #value = combo_dict.get(key)
        print(f"> MAP combo-{combo_labels['combo'+str(key+1)]}:\t {value}")
        if value > maxp:
            difference=value - maxp
            old_maxp=maxp
            maxp=value
            old_name=max_name
            old_isCombo=max_isCombo
            max_name=combo_labels['combo'+str(key+1)]
            max_isCombo=True
            old_key=max_key
            max_key=key
        elif value > old_maxp:
            difference=maxp - value
            old_maxp=value
            old_name=combo_labels['combo'+str(key+1)]
            old_key=key
           
    if not check_if_present(to_be_plotted, (max_key, max_isCombo)):
        to_be_plotted.append((max_key, max_isCombo))
    
    if not old_key is None and not check_if_present(to_be_plotted, (old_key, old_isCombo)):
        to_be_plotted.append((old_key, old_isCombo))
    
    
    print(f"Max MAP is {maxp} from {max_name}({'COMBO' if max_isCombo else 'MERGED'}) with difference from previous max of {difference} from {old_name}({'COMBO' if old_isCombo else 'MERGED'}) \n")
        

RANGE (0, 3)
> MAP combo-icb, icf, p3a:	 0.06474931369262016
> MAP combo-cb, icf, rp3b:	 0.06500029236303792
> MAP combo-cb, rp3b, p3a:	 0.06363190600262157
> MAP combo-icb, ucf, p3a:	 0.06510243113239321
> MAP combo-icb, ucf, rp3b:	 0.06803465666239525
> MAP combo-icb, icfcb, icf:	 0.0664798777960184
> MAP combo-icb, icfcb, p3a:	 0.06869084116414374
> MAP combo-icb, icfcb, rp3b:	 0.06877533145641564
> MAP combo-icb, icfcb, ucf:	 0.06909548333186183
> MAP combo-icb, icfcb, sbpr:	 0.06929929339228114
> MAP combo-icbsup, icfcb, icf:	 0.06963134653995043
> MAP combo-icbsup, icfcb, p3a:	 0.06775322798478764
> MAP combo-icbsup, icfcb, rp3b:	 0.06963275574616393
> MAP combo-icbsup, icfcb, ucf:	 0.06996848023410623
> MAP combo-icbsup, icfcb, sbpr:	 0.06853026425809057
> MAP combo-icbsup, icf, p3a:	 0.0708910268185396
> MAP combo-icbsup, icfcb, rp3b:	 0.06729558862627413
> MAP combo-icbsup, icf, ucf:	 0.06960873049906709
> MAP combo-icbsup, icf, sbpr:	 0.06539476428562532
> MAP combo-icbsup, p

KeyError: 'combo43'

In [ ]:
import matplotlib.pyplot as pyplot
%matplotlib inline  
pyplot.figure(figsize=(20,20))
for key, isCombo in to_be_plotted:
    l=[]
    if isCombo:
        for combo_dict in MAP_combo_per_group:
            l.append(combo_dict[key])
        style = '--o'
        label = combo_labels['combo'+str(key+1)]
    """
    else:
        for merged_dict in MAP_merged_per_group:
            l.append(merged_dict[key])
        style = '--x'
        label = merged_labels[key]
        """
    pyplot.plot(l, style, label = label)

pyplot.xticks(ticks=range(0,len(f_ranges)), labels=f_ranges)
pyplot.ylabel('MAP')
pyplot.xlabel('User Group')
pyplot.legend()
pyplot.show()

In [ ]:
f_ranges = [
    (0,3),
    (3,6),
    (6,12),
    (12,25),
    (25,-1)
]

for f_range in f_ranges:
    URM_test_group = parser.filter_URM_test_by_range(URM_list[0], URM_test_list[0], f_range)

---
### 5 balanced

In [ ]:
f_ranges = [
    (0,4),
    (4,7),
    (7,13),
    (13,26),
    (26,-1)
]

MAP_combo_per_group = []
cutoff = 10

for f_range in f_ranges:
    
    
    combo_dict={}
    for k in range (num_combo):
        r = []
        for i in range (len(URM_list)):
            URM_test_group = parser.filter_URM_test_by_range(URM_list[i], URM_test_list[i], f_range)
            evaluator_test = EvaluatorHoldout(URM_test_group, cutoff_list=[cutoff])
            if not isinstance(list_rec[k][0], list):
                rec = list_rec[k][0]
                try:
                    rec = rec(URM_all, ICM_all, verbose = False)
                except:
                    rec = rec(URM_all, verbose=False)
                try:
                    rec.load_model(f'stored_recommenders/seed_{str(seed)}_hybrid_search/{rec.RECOMMENDER_NAME}/', f'{str(seed)}_fold-{str(i)}')
                    print(f"{rec.RECOMMENDER_NAME} loaded. [seed={seed}, fold={i}]")
                except:
                    print(f"Fitting {rec.RECOMMENDER_NAME} ... [seed={seed}, fold={i}]")
                    rec.fit(**list_rec[k][1])
                    print(f"done.")
                    rec.save_model(f'stored_recommenders/seed_{str(seed)}_hybrid_search/{rec.RECOMMENDER_NAME}/', f'{str(seed)}_fold-{str(i)}')
            else:
                rec = HybridCombinationSearchCV(URM_list[i], ICM_all, list_rec[k][0], seed = seed, fold = i)
                rec.fit(**list_rec[k][1])

            results, _ = evaluator_test.evaluateRecommender(rec)
            r.append(results[cutoff]["MAP"])
        combo_dict[k]=np.average(r)

    MAP_combo_per_group.append(combo_dict)

In [ ]:
to_be_plotted = []
for i in range(0, len (f_ranges)):
    print(f"RANGE {f_ranges[i]}")
    maxp = 0
    difference = 0
    old_name = ''
    max_name=''
    max_isCombo=None
    old_isCombo=None
    max_key=None
    old_key=None
    
    combo_dict = MAP_combo_per_group[i]
    for key, value in combo_dict.items():
        #value = combo_dict.get(key)
        print(f"> MAP combo-{combo_labels['combo'+str(key+1)]}:\t {value}")
        if value > maxp:
            difference=value - maxp
            maxp=value
            old_name=max_name
            old_isCombo=max_isCombo
            max_name=combo_labels['combo'+str(key+1)]
            max_isCombo=True
            old_key=max_key
            max_key=key
        elif value > old_maxp:
            difference=maxp - value
            old_maxp=value
            old_name=combo_labels['combo'+str(key+1)]
            old_key=key
           
    if not check_if_present(to_be_plotted, (max_key, max_isCombo)):
        to_be_plotted.append((max_key, max_isCombo))
    
    if not old_key is None and not check_if_present(to_be_plotted, (old_key, old_isCombo)):
        to_be_plotted.append((old_key, old_isCombo))
    
    
    print(f"Max MAP is {maxp} from {max_name}({'COMBO' if max_isCombo else 'MERGED'}) with difference from previous max of {difference} from {old_name}({'COMBO' if old_isCombo else 'MERGED'}) \n")
        

In [ ]:
import matplotlib.pyplot as pyplot
%matplotlib inline  
pyplot.figure(figsize=(20,20))
for key, isCombo in to_be_plotted:
    l=[]
    if isCombo:
        for combo_dict in MAP_combo_per_group:
            l.append(combo_dict[key])
        style = '--o'
        label = combo_labels['combo'+str(key+1)]
    """
    else:
        for merged_dict in MAP_merged_per_group:
            l.append(merged_dict[key])
        style = '--x'
        label = merged_labels[key]
        """
    pyplot.plot(l, style, label = label)

pyplot.xticks(ticks=range(0,len(f_ranges)), labels=f_ranges)
pyplot.ylabel('MAP')
pyplot.xlabel('User Group')
pyplot.legend()
pyplot.show()